# Step 1: Motion Correction

Typically, one of the biggest sources of variance in the fMRI timeseries is due to subjects moving in the scanner. Subject motion is inevitable, as it is very difficult to stay perfectly still for the 5-10 minute duration of a typical fMRI task. Unfortunately, in cases where motion is particularly bad it can cause such damage to the data that the scan cannot be used. Even in cases where the motion seems slight, changes in head position of as little as 2-3mm can corrupt the data. Motion is therefore a serious issue for the quality of individual scans, but is also a big problem for group comparisons. For instance, if you have one group with less tolerance for the scanning environment than the other, such as the elderly or young children. Because motion can cause spurious activations to appear, one group moving more than the other may cause spurious group differences that we misconstrue as true neuronal differences between the groups. As such, we always have to encourage our subjects to stay as still as possible. Eliminating motion at the source is really the best way to deal with this issue. However, we also need to recognise that there will always be some motion, and need be aware of what we can and cannot do to try and fix it.

## Problems with motion

There are two main issues that motion can cause. The first is that motion causes individual voxels to suddenly contain data from a different part of the brain. The second is that motion can disrupt that actual signal recording from the scanner. Both these issues can be further compounded by motion that occurs more during one specific conditions of a task compared to others. In this case, the motion is said to be correlated with the task, which can create a serious confound in the data.

### Corruption of the timeseries

The main issue with motion is that if the subject moves then suddenly our timeseries at an individual voxel is no longer a continuous measure of that point in the brain. Instead, the timeseries measures several points in the brain over time. Depending on the spatial extent of the activity, and how small the voxel size is, this may or may not make a big difference. However, this becomes more problematic at tissue boundaries. For instance, when motion causes a voxel to contain signal from inside and outside the head, or inside and outside the ventricles. In these cases, the sudden shift in tissue type will cause a sudden shift in signal intensity that may be appear as true brain activity when we analyse the data. This is shown in {numref}`mot-spikes-fig` where 20 volumes in the middle of a functional series have been artificially shifted by the width of a voxel to the left and then back again. A voxel on the edge of the ventricles has been highlighted. We can see how the time series is corrupted by the motion. This is such an extreme change in intensity that it could be easily mistaken for true brain activity.

```{figure} images/mot-spikes.png
---
width: 800px
name: mot-spikes-fig
---
Illustration of how sudden changes in signal intensity can occur after head motion.
```

One saving grace of this form of motion is that often we will see activation effects that sit along the border between tissue types. So any activation pattern that sits along the edge of the brain, or along the edge of the ventricles, should be treated with extreme caution. An example is shown in {numref}`mot-edge-fig`. That being said, we cannot rely on motion artifacts always being visible in our results, as motion can also cause more subtle corrupting effects on the data. As we will see below, there are means for correcting motion, but these are imperfect and have their limitations. In some cases the motion can be so bad that we simply cannot trust the data and thus have to just throw the scan away.

```{figure} images/mot-edge.png
---
width: 250px
name: mot-edge-fig
---
Illustration of how motion artefacts can appear along the edges of the brain.
```

### Corruption of the measurement of the signal

A more insidious effect of motion is that it can impact the signal that is measured by the scanner. When the subject's head moves, protons from a neighbouring slice with a different excitation move into the voxel. This disrupts the magnetic gradients and the subsequent reconstructed signal will not be accurate. This is exacerbated when using an inter-leaved acquisition sequence, as the excitation differences between neighbouring slices will be greater. The important point here is that this cannot be corrected by the standard motion correction techniques described below. Although so-called spin-history correction has been proposed (e.g. Friston et al, 1996), this is not in common use and any residual effects of motion after pre-processing are usually accommodated during the statistical modelling. However, the success of this approach remains somewhat unknown and so this issue remains one that is somewhat ignored during standard fMRI processing. In extreme cases, the spin-history effects will be easily visible as striping in the image, as shown in {numref}`spin-hist-fig`. The top row shows an interleaved sequence and the bottom row shows a sequential sequence. The left column is a scan without motion and the right is a scan with motion. So the lesson here is to make sure you check your data visually to see whether there are any extreme spin history effects. If there are, the scan may be unusable.

```{figure} images/spin-history.gif
---
width: 500px
name: spin-hist-fig
---
Spin history effects in both an interleaved (top) and sequential (bottom) sequence.
```

### Motion correlated with the task

The point where motion-based corruption of the timeseries becomes especially problematic is when it happens more in some conditions of the scanner task than others. For instance, consider a task with alternating easy and difficult conditions. It is plausible that frustration during the difficult condition may cause subjects to move more compared to the easy condition. If this happens, we will get movement-related intensity changes during the difficult condition that will be inseparable from intensity changes that related to true brain activity. At which point, we have no idea whether results for the difficult condition represent motion, true brain activity, or some messy combination of the two. To make matters worse, any attempt to remove the motion effects will also remove true brain activity, because there is no way to separate them. At this point, motion becomes an inescapable confound and you cannot reach any conclusions about the data. There are ways to check for the correlation between the task and motion, but if this correlation if high then there is nothing that can be done to correct for it and the data become fundamentally unreliable.

## Motion correction theory

Now we know the problems caused by motion, what can we do about it? Typically we perform spatial motion correction as a first pre-processing step, to move all the volumes back into alignment with one another. In principle, this will alleviate the issues of the timeseries being corrupted, but of course will not fix spin-history effects nor issues of task-correlated motion. So it is not a complete solution to motion, but will help with some of the more common issues. In SPM, this form of motion correction is known as realignment and is illustrated in {numref}`realign-poldrack-fig` from Poldrack, Mumford and Nichols (2011).

```{figure} images/realign-poldrack.png
---
width: 800px
name: realign-poldrack-fig
---
Illustration of motion correction.
```

At its most basic, spatial motion correction is an image registration problem. We choose a single fMRI volume to act as our stationary reference image and then align all the other volumes to match. As such, you can think of it as a series of registration steps where the reference stays the same and the source is changed each time to be a subsequent fMRI volume. From this perspective, there is nothing particularly special about the mechanics of motion correction. The main focus of motion correction algorithms is speed , because we need to align 200+ images together and do not want to sit around for days waiting for the computer to finish. As such, most motion correction algorithms use least-squares as the cost function. In addition, motion correction only uses 6 parameters because scaling and shearing are not relevant for motion correction. Finally, SPM uses a very specific optimisation algorithm when estimating the transformation that can converge on a solution very quickly (as described in Friston et al. (2005)). Of note, however, is that this algorithm is more accurate for small movements and will not be able to correct very large motions accurately. So, we can think of motion correction as a highly tuned version of image registration, but with some caveats and limitations.

### Motion parameters

One of the main outputs from motion correction is the registration parameters for each volume. As we know, registration involves estimating a matrix T that is used to transform the source to match the target. For motion correction, there will be a T matrix estimated for each volume, that will contain the translations and rotations needed to match that volume to the reference. The parameters of these $\mathbf{T}$ matrices are useful because they tell us how much the subject moved over time. For instance, extracting and then plotting the $x$-translations from each $\mathbf{T}$ matrix allows us to see how much the subject deviated from the target volume along the $x$-axis over time. An example of the form of motion parameter plot produced by motion correction is shown below.

```{figure} images/mot-params.png
---
width: 650px
name: mot-params-fig
---
Example of the plots of motion parameters produced after motion correction.
```

### Resampling the fMRI data

Once all the T matrices have been estimated it would be typical to turn these world-space transformations into voxel-space transformations, so that the timeseries at each voxel represents motion-corrected data. Resampling at this stage is particularly important for slice-timing correction, which is our next pre-processing step. This is because we need to make sure we do not have bits of the brain moving in and out of neighbouring slices across the course of the scan. In terms of the interpolation, higher-order B-spline methods are generally preferred, as evidence suggests that linear methods can introduce large errors (Ostuni et al., 1997). In principle we could just choose the highest-order available to us, but the gains in accuracy are likely going to be undone once we get to the final pre-processing step, which is spatial smoothing. SPM therefore defaults to a 4th-degree B-spline as a compromise.

### How much motion it too much?

Considering the fact that motion is inevitable, but also damaging, how much motion would we allow in our data before deciding the scan was too corrupted to use? This seems like an obvious question, but it is actually difficult to answer. As mentioned earlier, volume-to-volume motion is more damaging than steady and slow motion, because it is more difficult for traditional spatial motion correction to deal with. According to Poldrack, Mumford and Nichols (2011), a sudden translation spike of more than 1/2 a voxel size should be cause for concern. So if the voxels were 3mm isotropic then a spike of > 1.5mm in the x, y or z planes may be a concern. This is similar for rotations, where you can calculate the angle for a distance of 1.5mm using

$$
\theta = d\frac{180}{\pi r}
$$

where $d$ is the distance and $r$ is the radius of a circle. If we consider the head as a circle, then we can use a rough radius of 50mm (Power et al., 2012). This means for our 3mm isotropic voxels we have

$$
1.72 = 1.5 \times \frac{180}{\pi \times 50}
$$

so that any sudden rotation greater than 1.72$^{\circ}$ are also cause for concern. So what can be done about this? To some extent, it depends upon how corrupted an individual scan is. A couple of spikes may suggest the data is recoverable, whereas a scan that is littered with motion spikes will almost certainly be a lost cause. As an example, the plot below shows a couple of concerning spikes in the z-translation trace, but although these are large this would not be enough to suggest getting rid of the scan. However, if similar spikes were across the whole series of scans, we would begin to be concerned with the quality of the data and may need to consider not using this scan in our analysis.

## Motion correction in SPM

The video below will show you how to perform realignment using SPM12. The default in SPM is to perform a two-pass procedure where the images are aligned to the first volume, then a mean image is created from the realigned volumes and a second registration is performed to align these volumes with the mean. The final T matrices represent a combination of the transformations needed for both steps. The idea is that the registration can be more finely tuned using this second pass, although it realistically does not make a huge difference. Importantly, we will be saving this mean image to use in a later pre-processing step.

<div style="max-width: 1280px"><div style="position: relative; padding-bottom: 56.25%; height: 0; overflow: hidden;"><iframe src="https://livemanchesterac.sharepoint.com/sites/UOM-FBMH-MSc-NCCN/_layouts/15/embed.aspx?UniqueId=27a69ee8-454c-4fe5-a640-e172328d949f&embed=%7B%22hvm%22%3Atrue%2C%22ust%22%3Atrue%7D&referrer=StreamWebApp&referrerScenario=EmbedDialog.Create" width="1280" height="720" frameborder="0" scrolling="no" allowfullscreen title="3.Motion-Correction.mov" style="border:none; position: absolute; top: 0; left: 0; right: 0; bottom: 0; height: 100%; max-width: 100%;"></iframe></div></div>